In [6]:
import pandas
import altair as alt
import pandas.io.json
import swifter
# import dask.dataframe
# from dask.distributed import Client

# https://github.com/pandas-dev/pandas/issues/26068#issuecomment-550900208
import json
pandas.io.json._json.loads = lambda s, *a, **kw: json.loads(s)

# dask.config.set({'temporary_directory': '/tmp'})
alt.data_transformers.enable('data_server')
pandas.set_option('max_colwidth', 200)

# client = Client()
# client

In [7]:
# data = dask.dataframe.read_json(
#     'skimage.jsonl',
#     blocksize=1000 * 1024,
#     meta=pandas.DataFrame.from_dict({"function": ["str"], "params": [{}]})
# )
# data
data = pandas.read_json('data/raw/skimage.jsonl', lines=True)

In [51]:
list.__name__

'list'

In [53]:
operator.inv

'inv'

In [64]:
import re

In [85]:
reg.match(operator.getitem.__doc__)[1]

'a[b]'

In [77]:
m[1]

'a[b] = c'

In [61]:
s = operator.setitem.__doc__

In [ ]:
s.strip()

In [87]:
int("d")

ValueError: invalid literal for int() with base 10: 'd'

In [141]:
sorted({1, 2, 3})

[1, 2, 3]

In [167]:
import dataclasses
import typing
import collections
from __future__ import annotations

In [161]:
s = set()

In [187]:
s = {1}

In [188]:
s.update([1, 2, 3])

In [189]:
s

{1, 2, 3}

In [195]:
reg = re.compile(r"Same\ as\ ([^.(]+)")

def get_operator_format(o):
    return reg.match(o.__doc__)[1]

@dataclasses.dataclass(frozen=True)
class GenericType:
    name: str
    args: typing.Tuple[typing.Union[str, GenericType, UnionType], ...]
    
    def __str__(self):
        return f"{self.name}[{', '.join(map(str, self.args))}]"

    def __repr__(self):
        return str(self)

@dataclasses.dataclass(frozen=True)
class UnionType:
    args: typing.Tuple[typing.Union[str, GenericType], ...]
    
    def __str__(self):
        return f"Union[{', '.join(map(str, self.args))}]"

    def __repr__(self):
        return str(self)
    
def unify(types):
    types = set(types)
    if len(types) == 1:
        return types.pop()
    if not types:
        return 'object'
    # mapping of generic type name and n args to args
    generic_types = collections.defaultdict(set)
    regular_types = []
    while types:
        t = types.pop()
        if isinstance(t, GenericType):
            generic_types[(t.name, len(t.args))].add(t.args)
        elif isinstance(t, UnionType):
            types.update(t.args)
        else:
            regular_types.append(t)
    
    for keys, args in generic_types.items():
        regular_types.append(GenericType(keys[0], tuple(map(unify, zip(*args)))))
    
    if len(regular_types) == 1:
        return regular_types[0]
    return UnionType(tuple(sorted(regular_types)))

def t(v):
    """
    Pretty print type of value
    """
    if isinstance(v, dict):
        tp = v['t']
        if tp == 'tuple' and v['v']:
            return GenericType('tuple', tuple(map(t, v["v"])))
        if tp == 'builtins.module':
            return v['v']
        if tp == 'builtins.type':
            return GenericType('Type', (v["v"],))
        return tp
    if isinstance(v, list) and v:
        return GenericType("list", (unify(map(t, v)),))
    return type(v).__name__

def pretty_print(row):
    """
    Pretty prints a row. Function call is like:
        
        fn_name(FirstArgType, SecondArgType, kw=KwArgType)
    
    getattr is
    
        Type.attr
    
    getitem is
        
        ObjectType[IndexType]
    
    setitem is
        
        ObjectType[IndexType] = ValueType
    
    binary ops are
    
        LeftType * RightType
        etc..
    
    iter is
    
        *Type
    
    inplace binary ops are
    
        Type += OtherType
        etc
    
    setatr is
        
        Type.atrr = AssignType
    """
    f = row['function']
    p = row['params']
    if f == 'builtins.getattr':
        return f"{t(p['0'])}.{p['1']}"
    if f == 'builtins.setattr':
        return f"{t(p['obj'])}.{p['name']} = {t(p['value'])}"
    if f == 'builtins.iter':
        return f"*{t(p['0'])}"
    elif f.startswith("_operator"):
        op = getattr(operator, f[len("_operator."):])
        # special case cotains b/c we ordered wrong
        format_str = get_operator_format(op) if op is not operator.contains else 'a in b'
        for arg, val in p.items():
            format_str = format_str.replace(arg, str(t(val)))
        return format_str
    args = {}
    kwargs = {}
    for key, value in p.items():
        try:
            idx = int(key)
        except ValueError:
            kwargs[key] = t(value)
        else:
            args[idx] = t(value)
    return f"""{f}({', '.join(
        [str(args[i]) for i in range(len(args))] +
        [f'{key}={kwargs[key]}' for key in sorted(kwargs.keys())])
    })"""

def pretty_print_untyped(row):
    f = row['function']
    p = row['params']
    if f == 'builtins.getattr':
        return f"{t(p['0'])}.{p['1']}"
    if f == 'builtins.setattr':
        return f"{t(p['obj'])}.{p['name']} ="
    elif f.startswith("_operator"):
        return None
    return str(f)


In [129]:
examples.to_dict('records')

[{'function': '_operator.add',
  'location': '/Users/saul/Downloads/scikit-image-0.17.0/skimage/color/colorconv.py:273',
  'params': {'a': 2.0, 'b': {'t': 'numpy.ndarray', 'v': [[1180], 'float64']}}},
 {'function': '_operator.and_',
  'location': '/Users/saul/Downloads/scikit-image-0.17.0/skimage/feature/brief.py:159',
  'params': {'a': {'t': 'numpy.ndarray', 'v': [[64], 'bool']},
   'b': {'t': 'numpy.ndarray', 'v': [[64], 'bool']}}},
 {'function': '_operator.contains',
  'location': '/Users/saul/Downloads/scikit-image-0.17.0/skimage/util/dtype.py:248',
  'params': {'a': {'t': 'builtins.type', 'v': 'numpy.uint8'},
   'b': [{'t': 'builtins.type', 'v': 'numpy.bool_'},
    {'t': 'builtins.type', 'v': 'numpy.float16'},
    {'t': 'builtins.type', 'v': 'numpy.float32'},
    {'t': 'builtins.type', 'v': 'numpy.float64'},
    {'t': 'builtins.type', 'v': 'numpy.int8'},
    {'t': 'builtins.type', 'v': 'numpy.uint8'},
    {'t': 'builtins.type', 'v': 'numpy.int16'},
    {'t': 'builtins.type', 'v': 

In [196]:
list(map(pretty_print_untyped, examples.to_dict('records')))

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'numpy.linspace',
 'builtins.iter',
 'numpy.ndarray.flat =',
 'numpy.absolute',
 'numpy.add',
 'numpy.all',
 'numpy.allclose',
 'numpy.alltrue',
 'numpy.amax',
 'numpy.amin',
 'numpy.angle',
 'numpy.any',
 'numpy.apply_along_axis',
 'numpy.apply_over_axes',
 'numpy.arange',
 'numpy.arcsin',
 'numpy.arctan',
 'numpy.arctan2',
 'numpy.argmax',
 'numpy.argmin',
 'numpy.argsort',
 'numpy.around',
 'numpy.array',
 'numpy.array2string',
 'numpy.array_equal',
 'numpy.asanyarray',
 'numpy.asarray',
 'numpy.ascontiguousarray',
 'numpy.asfortranarray',
 'numpy.atleast_1d',
 'numpy.atleast_2d',
 'numpy.atleast_3d',
 'numpy.bincount',
 'numpy.can_cast',
 'numpy.cbrt',
 'numpy.ceil',
 'numpy.choose',
 'numpy.clip',
 'numpy.column_stack',
 'numpy.concatenate',
 'numpy.conjugate',
 'numpy.convolve',


In [193]:
list(map(pretty_print, examples.to_dict('records')))

['float + numpy.ndarray',
 'numpy.ndarray & numpy.ndarray',
 'Type[numpy.uint8] in list[Type[Union[numpy.bool_, numpy.float16, numpy.float32, numpy.float64, numpy.int16, numpy.int32, numpy.int64, numpy.int8, numpy.longlong, numpy.uint16, numpy.uint32, numpy.uint64, numpy.uint8, numpy.ulonglong]]]',
 'numpy.ndarray == numpy.ndarray',
 'numpy.int64 // int',
 'numpy.ndarray >= float',
 'numpy.ndarray[builtins.slice]',
 'numpy.float64 > float',
 'numpy.ndarray += numpy.ndarray',
 'numpy.ndarray &= numpy.ndarray',
 'numpy.ndarray //= int',
 'numpy.float64 %= float',
 'numpy.ndarray *= numpy.ndarray',
 '~numpy.ndarray',
 'numpy.ndarray |= numpy.ndarray',
 'numpy.ndarray -= int',
 'numpy.ndarray /= float',
 'numpy.float64 <= float',
 'numpy.ndarray @ numpy.ndarray',
 'numpy.ndarray % int',
 'numpy.ndarray * numpy.ndarray',
 'numpy.int64 != int',
 '-numpy.ndarray',
 'not numpy.bool_',
 'numpy.ndarray | numpy.ndarray',
 'numpy.ndarray ** int',
 'numpy.ndarray[numpy.ndarray] = numpy.ndarray',
 '

In [47]:
examples = data.groupby('function').first().reset_index()

In [48]:
examples[~examples['function'].str.startswith('numpy')]

,function,location,params
0,_operator.add,/Users/saul/Downloads/scikit-image-0.17.0/skimage/color/colorconv.py:273,"{'a': 2.0, 'b': {'t': 'numpy.ndarray', 'v': [[1180], 'float64']}}"
1,_operator.and_,/Users/saul/Downloads/scikit-image-0.17.0/skimage/feature/brief.py:159,"{'a': {'t': 'numpy.ndarray', 'v': [[64], 'bool']}, 'b': {'t': 'numpy.ndarray', 'v': [[64], 'bool']}}"
2,_operator.contains,/Users/saul/Downloads/scikit-image-0.17.0/skimage/util/dtype.py:248,"{'a': {'t': 'builtins.type', 'v': 'numpy.uint8'}, 'b': [{'t': 'builtins.type', 'v': 'numpy.bool_'}, {'t': 'builtins.type', 'v': 'numpy.float16'}, {'t': 'builtins.type', 'v': 'numpy.float32'}, {'t'..."
3,_operator.eq,/Users/saul/Downloads/scikit-image-0.17.0/skimage/_shared/_geometry.py:34,"{'a': {'t': 'numpy.ndarray', 'v': [[2], 'float64']}, 'b': {'t': 'numpy.ndarray', 'v': [[2], 'float64']}}"
4,_operator.floordiv,/Users/saul/Downloads/scikit-image-0.17.0/skimage/exposure/_adapthist.py:262,"{'a': {'t': 'numpy.int64'}, 'b': 128}"
5,_operator.ge,/Users/saul/Downloads/scikit-image-0.17.0/skimage/color/delta_e.py:216,"{'a': {'t': 'numpy.ndarray', 'v': [[34], 'float64']}, 'b': 6.283185307179586}"
6,_operator.getitem,/Users/saul/Downloads/scikit-image-0.17.0/skimage/_shared/_geometry.py:55,"{'a': {'t': 'numpy.ndarray', 'v': [[4], 'int64']}, 'b': {'t': 'builtins.slice', 'v': [None, -1, None]}}"
7,_operator.gt,/Users/saul/Downloads/scikit-image-0.17.0/skimage/_shared/utils.py:235,"{'a': {'t': 'numpy.float64', 'v': 0.09999999999999964}, 'b': 0.5}"
8,_operator.iadd,/Users/saul/Downloads/scikit-image-0.17.0/skimage/filters/edges.py:181,"{'a': {'t': 'numpy.ndarray', 'v': [[103, 103], 'float64']}, 'b': {'t': 'numpy.ndarray', 'v': [[103, 103], 'float64']}}"
9,_operator.iand,/Users/saul/Downloads/scikit-image-0.17.0/skimage/feature/peak.py:39,"{'a': {'t': 'numpy.ndarray', 'v': [[512, 512, 5], 'bool']}, 'b': {'t': 'numpy.ndarray', 'v': [[512, 512, 5], 'bool']}}"


In [26]:
data['params_str'] = data['params'].to_string()

KeyboardInterrupt: 

In [11]:
s = data.head(n=100)

In [25]:
s['params'].to_string()

"0                                                                                                                                              {'0': {'t': 'builtins.module', 'v': 'numpy'}, '1': 'linspace'}\n1                                                                                                                        {'start': -5, 'stop': 0, 'num': 5000, 'endpoint': {'t': 'builtins.bool', 'v': True}}\n2                                                                                                                                                     {'0': {'t': 'numpy.ndarray', 'v': [[5000], 'float64']}}\n3                                                                                                                                                     {'0': {'t': 'numpy.ndarray', 'v': [[5000], 'float64']}}\n4     {'0': [0.0067521147429943085, 0.006757751107215881, 0.006763387471437454, 0.006769023835659027, 0.0067746601998806, 0.006780296564102173, 0.006785932928323746, 0.006

In [17]:
ops = data.groupby(['function'])['location'].nunique()

In [28]:
ops_df = pandas.DataFrame({"count": ops}).reset_index()

In [30]:
ops_df

,function,count
0,_operator.add,404
1,_operator.and_,33
2,_operator.contains,42
3,_operator.eq,529
4,_operator.floordiv,20
...,...,...
296,numpy.unravel_index,13
297,numpy.vstack,21
298,numpy.where,22
299,numpy.zeros,731


In [33]:
ops_df[~ops_df['function'].str.startswith('numpy')]

,function,count
0,_operator.add,404
1,_operator.and_,33
2,_operator.contains,42
3,_operator.eq,529
4,_operator.floordiv,20
5,_operator.ge,76
6,_operator.getitem,1680
7,_operator.gt,207
8,_operator.iadd,119
9,_operator.iand,2


In [164]:
list(zip((1, 2, 3), (4, 5, 6), (7, 8, 9)))

[(1, 4, 7), (2, 5, 8), (3, 6, 9)]

In [35]:
import operator

In [42]:
operator.invert?

Signature: operator.invert(a, /)
Docstring: Same as ~a.
Type:      builtin_function_or_method


In [197]:
alt.Chart(ops_df[~ops_df['function'].str.startswith('numpy')]).mark_bar().encode(
    y=alt.Y('function:N', sort='-x'),
    x=alt.X('count:Q', scale=alt.Scale(type='log')),
    tooltip=['function', 'count']
)

alt.Chart(...)

In [8]:
data.info()
data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7862630 entries, 0 to 7862629
Data columns (total 3 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   location  object
 1   function  object
 2   params    object
dtypes: object(3)
memory usage: 180.0+ MB


,location,function,params
0,/Users/saul/Downloads/scikit-image-0.17.0/skimage/_shared/tests/test_fast_exp.py:7,builtins.getattr,"{'0': {'t': 'builtins.module', 'v': 'numpy'}, '1': 'linspace'}"
1,/Users/saul/Downloads/scikit-image-0.17.0/skimage/_shared/tests/test_fast_exp.py:7,numpy.linspace,"{'start': -5, 'stop': 0, 'num': 5000, 'endpoint': {'t': 'builtins.bool', 'v': True}}"
2,/Users/saul/Downloads/scikit-image-0.17.0/skimage/_shared/tests/test_fast_exp.py:10,numpy.exp,"{'0': {'t': 'numpy.ndarray', 'v': [[5000], 'float64']}}"
3,/Users/saul/Downloads/scikit-image-0.17.0/skimage/_shared/tests/test_fast_exp.py:13,builtins.iter,"{'0': {'t': 'numpy.ndarray', 'v': [[5000], 'float64']}}"
4,/Users/saul/Downloads/scikit-image-0.17.0/skimage/_shared/tests/test_fast_exp.py:13,numpy.array,"{'0': [0.0067521147429943085, 0.006757751107215881, 0.006763387471437454, 0.006769023835659027, 0.0067746601998806, 0.006780296564102173, 0.006785932928323746, 0.006791569292545319, 0.006797205656..."
...,...,...,...
7862625,/Users/saul/Downloads/scikit-image-0.17.0/skimage/util/unique.py:41,builtins.getattr,"{'0': {'t': 'numpy.ndarray', 'v': [[4], 'uint8']}, '1': 'ndim'}"
7862626,/Users/saul/Downloads/scikit-image-0.17.0/skimage/util/tests/test_unique_rows.py:37,numpy.arange,{'0': 8}
7862627,/Users/saul/Downloads/scikit-image-0.17.0/skimage/util/tests/test_unique_rows.py:37,numpy.ndarray.reshape,"{'0': {'t': 'numpy.ndarray', 'v': [[8], 'int64']}, '1': {'t': 'tuple', 'v': [2, 2, 2]}}"
7862628,/Users/saul/Downloads/scikit-image-0.17.0/skimage/util/unique.py:39,builtins.getattr,"{'0': {'t': 'numpy.ndarray', 'v': [[2, 2, 2], 'int64']}, '1': 'ndim'}"


In [9]:
just_numpy = data[data.function.str.startswith("numpy.")]
counts = just_numpy.groupby('function').count()
counts['occurance'] = counts['params'] / counts['params'].sum()

In [10]:
alt.Chart(counts.reset_index()).mark_bar().encode(
    y=alt.Y('function:N', sort='-x'),
    x=alt.X('occurance:Q', axis=alt.Axis(format='%'), scale=alt.Scale(type='log')),
    tooltip=['function', 'occurance', 'params']
)

alt.Chart(...)

In [24]:
numpy_private = just_numpy[just_numpy.function.str.contains("._", regex=False)]


In [25]:
pandas.options.display.max_rows = 999

numpy_private.groupby(['function', 'location']).count()

,,params
function,location,
numpy.core._dtype._construction_repr,/usr/local/Caskroom/miniconda/base/envs/python-record-api/lib/python3.8/site-packages/numpy/core/_dtype.py:58,3
numpy.core._dtype._kind_name,/usr/local/Caskroom/miniconda/base/envs/python-record-api/lib/python3.8/site-packages/numpy/core/_dtype.py:344,51
numpy.core._dtype._name_includes_bit_suffix,/usr/local/Caskroom/miniconda/base/envs/python-record-api/lib/python3.8/site-packages/numpy/core/_dtype.py:347,51
numpy.core._multiarray_umath.implement_array_function,<__array_function__ internals>:5,2971
numpy.core.fromnumeric._all_dispatcher,<__array_function__ internals>:4,9
numpy.core.fromnumeric._amax_dispatcher,<__array_function__ internals>:4,46
numpy.core.fromnumeric._amin_dispatcher,<__array_function__ internals>:4,47
numpy.core.fromnumeric._prod_dispatcher,<__array_function__ internals>:4,854
numpy.core.fromnumeric._shape_dispatcher,<__array_function__ internals>:4,853


In [26]:
getattr = get_args('builtins.getattr')

/usr/local/Caskroom/miniconda/base/envs/python-record-api/lib/python3.8/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [23]:
getattr

,0,1
function,,
builtins.getattr,builtins.module,'byte'
builtins.getattr,builtins.module,'ubyte'
builtins.getattr,builtins.module,'short'
builtins.getattr,builtins.module,'ushort'
builtins.getattr,builtins.module,'intc'
...,...,...
builtins.getattr,builtins.module,'uint8'
builtins.getattr,numpy.ndarray,'ndim'
builtins.getattr,numpy.ndarray,'ndim'


In [24]:
alt.Chart(getitem.groupby('b').count().reset_index()).mark_bar().encode(
    y=alt.Y('b:N', sort='-x'),
    x=alt.X('a:Q', scale=alt.Scale(type='log')),
    tooltip=['b', 'a']
)

alt.Chart(...)

In [24]:
def stringify(v):
    """
    Turn a complex type into a simpler string, yes with less data, but easier to group by.
    """
    if isinstance(v, int):
        return "int"
    if isinstance(v, str):
        return str(repr(v))
    if isinstance(v, float):
        return "float"
    if isinstance(v, list):
        return f"List[{stringify(v[0]) if v else 'Any'}]"
    if v is None:
        return 'None'
    t = v['t']
    if t == 'builtins.type':
        return f"Type[{v['v']}]"
    if t == 'builtins.module':
        return v['v']
    if t == 'tuple':
        return f"Tuple[{', '.join(stringify(a) for a in v['v'])}]"
    return t
        

In [25]:
def get_args(fn_name):
    return data[data.index == fn_name].swifter.apply(
        lambda x: pandas.Series({k: stringify(v) for k, v in x['params'].items()}),
        axis=1
    )

In [10]:
def make_arg_chart(df):
    selections = [alt.selection_multi(fields=[col]) for col in df.columns]
    charts = []
    for i, col in enumerate(df.columns):
        new_chart = alt.Chart(df).mark_bar().encode(
            alt.Y(col, sort='-x', type='nominal'),
            alt.X('count()', scale=alt.Scale(type='log')),
            color=alt.condition(selections[i], alt.value('lightblue'), alt.value('lightgray'))
        ).add_selection(
            selections[i]
        )
        for other_i, selection in enumerate(selections):
            # don't filter on my own selection
            if other_i == i:
                continue
            new_chart = new_chart.transform_filter(selection)
        charts.append(new_chart)
    return alt.concat(*charts, columns=3)

In [11]:
def inspect_functionn(fn_name):
    df = get_args(fn_name)
    return make_arg_chart(df)